In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.feature_selection import mutual_info_classif

from src.core.distribution_data import *
from src.core import file_manager as fm
from src.embeddings.constants import EMBEDDING_MODELS_TRANSLATION

In [3]:
variation = 'without_others_intent/k100_without_sentences_higher_than_median'
work_dir = fm.filename_from_data_dir( f'output/patient/{variation}')

In [4]:
def print_dependencies_blox_pot(file_name, is_json_collumn=True):
  models = []
  mi = []

  for key in EMBEDDING_MODELS_TRANSLATION:
    print(f'Processing mi for {key}')
    df = pd.read_csv(f'{work_dir}/{key}/{file_name}')
    
    intent_indexes = {key: index for index, key in enumerate(sorted(df['intent'].unique()))}
    df['intent_indexes'] = df['intent'].map(intent_indexes)

    labels = df['intent_indexes']

    if 'embeddings' not in df:
      df = fm.add_embeddings(df, key)
    
    embeddings = np.array(df['embeddings'].apply(
      lambda x: np.array(json.loads(x)[0] if is_json_collumn else x[0])
      ).to_list())

    dependencies = mutual_info_classif(embeddings, labels, n_neighbors=3)

    models.extend([EMBEDDING_MODELS_TRANSLATION[key]] * len(dependencies))
    mi.extend(dependencies)

  data = pd.DataFrame({'Embedding Models': models, 'Mutual Information Score': mi})
  
  fig = px.box(data, x='Embedding Models', y='Mutual Information Score')

  fig.update_layout(title='Mutual Information distribution of each Embedding Model')

  fig.show()

In [5]:
df = pd.read_csv(f'{work_dir}/bert_pt/training_data.csv')

df

,txt,annotated_txt,label,distance,intent,embeddings
0,Na verdade é minha companheira que está com es...,Na verdade é minha companheira que está com es...,34,0.132112,inform_symptoms,"[[-0.20533463000000002, -0.48095354, 0.5711158..."
1,Estou tossindo ainda,Estou [tossindo](SINTOMA) ainda,11,0.203283,inform_symptoms,"[[-0.022659639000000002, -0.12153876, -0.04113..."
2,Tava muito lotada upa fiquei c medo voltei p casa,Tava muito lotada upa fiquei c medo voltei p casa,37,0.179013,inform_symptoms,"[[-0.06520815, -0.4126552, 0.01532718000000000..."
3,E ela teve contato,E ela teve contato,38,0.282432,inform_symptoms,"[[-0.12014482, -0.021971945, 0.682200599999999..."
4,mas agora tento não ver mais,mas agora tento não ver mais,70,0.189368,inform_medicine,"[[0.065612495, -0.47582957000000004, 0.2969232..."
...,...,...,...,...,...,...
5970,Não consigo fazer praticamente nada,Não consigo fazer praticamente nada,70,0.190272,inform_medicine,"[[0.31433326, -0.36428043, 0.09387486, 0.21204..."
5971,"Em relação a dor, não estou tomando analgésico...","Em relação a [dor](SINTOMA), não estou tomando...",21,0.124493,inform_symptoms,"[[0.12616359, -0.41875008, 0.30475128, 0.09589..."
5972,"Se vc gratuito eu quero, pq eu não tenho condi...","Se vc gratuito eu quero, pq eu não tenho condi...",21,0.123241,inform_symptoms,"[[-0.033070005, -0.31333923, 0.4372873, -0.059..."
5973,Não sinto cheiro de nada,Não sinto cheiro de nada,54,0.156979,inform_symptoms,"[[0.41675076, -0.43822974000000003, -0.2246300..."


In [6]:
df = pd.read_csv(f'{work_dir}/bert_pt/annotated_sentences.csv')

df

,txt,annotated_txt,label,distance,intent
0,Tô muito nervosa Marcela com tudo isso,Tô muito nervosa Marcela com tudo isso,11,0.145944,inform_symptoms
1,Com muito medo,Com muito medo,64,0.271775,inform_symptoms
2,Não,Não,7,0.129577,greeting
3,Tem algum remédio que a pessoa tome para se a...,Tem algum remédio que a pessoa tome para se a...,6,0.176322,request_inform
4,Tá certo,Tá certo,84,0.113250,greeting
...,...,...,...,...,...
8832,"Quando ando , quando como e as vezes sem fazer...","Quando ando , quando como e as vezes sem fazer...",80,0.138620,inform_symptoms
8833,Quando tento eu canso logo,Quando tento eu canso logo,80,0.193989,inform_symptoms
8834,E fico com falta de ar,E fico com [falta de ar](SINTOMA),15,0.093503,inform_symptoms
8835,"Ñ, só pressão baixa","Ñ, só pressão baixa",65,0.191652,inform_symptoms


In [7]:
print_dependencies_blox_pot('training_data.csv')

Processing mi for bert_pt
Processing mi for flair_pt
Processing mi for glove
Processing mi for lasbe
Processing mi for use


In [14]:
print_dependencies_blox_pot('test_data.csv')

Processing mi for bert_pt
Processing mi for flair_pt
Processing mi for glove
Processing mi for lasbe
Processing mi for use


In [11]:
print_dependencies_blox_pot('annotated_sentences.csv', is_json_collumn=False)

Processing mi for bert_pt
Processing mi for flair_pt
Processing mi for glove
Processing mi for lasbe
Processing mi for use


In [18]:
anottated_manual_path = fm.filename_from_data_dir( f'output/patient/{variation}/intersection_300_sentences_with_label.csv')
anottated_manual_df = pd.read_csv(anottated_manual_path)
anottated_manual_df.describe()

,txt,intent,doubt
count,300,300,15
unique,300,5,5
top,Não faço uso de remédio. Estou tomando devido ...,inform_symptoms,/inform_symptoms
freq,1,182,7


In [16]:
def print_dependencies_blox_pot(anottated_manual_df):
  models = []
  mi = []

  for key in EMBEDDING_MODELS_TRANSLATION:
    print(f'Processing mi for {key}')

    df = anottated_manual_df.loc[anottated_manual_df['intent'] != 'others']

    df = fm.add_embeddings(df, key)

    intent_indexes = {key: index for index, key in enumerate(sorted(df['intent'].unique()))}
    df['intent_indexes'] = df['intent'].map(intent_indexes)

    labels = df['intent_indexes']
    embeddings = np.array(df['embeddings'].apply(lambda x: np.array(x[0])).to_list())

    dependencies = mutual_info_classif(embeddings, labels, n_neighbors=3)

    models.extend([EMBEDDING_MODELS_TRANSLATION[key]] * len(dependencies))
    mi.extend(dependencies)

  data = pd.DataFrame({'Embedding Models': models, 'Mutual Information': mi})

  fig = px.box(data, x='Embedding Models', y='Mutual Information')

  fig.update_layout(
    font=dict(size=22)
  )

  fig.show()

In [19]:
print_dependencies_blox_pot(anottated_manual_df)

Processing mi for bert_pt
Processing mi for flair_pt
Processing mi for glove
Processing mi for lasbe
Processing mi for use
